# BQ Cost Optimization Assessment

## Summary
The purpose of this notebook is to summarize BigQuery pricing for your customer and Google to process SQL queries needed to monitor BigQuery cost and performance.

## BigQuery Editions
There are 2 pricing strategies for BigQuery
On-demand
Capacity

### On-demand
By default, queries are billed using the on-demand (per TiB) pricing model, where you pay for the data scanned by your queries.

With on-demand pricing, you will generally have access to up to 2,000 concurrent slots, shared among all queries in a single project. Periodically, BigQuery will temporarily burst beyond this limit to accelerate smaller queries. In addition, you might occasionally have fewer slots available if there is a high amount of contention for on-demand capacity in a specific location.

In Europe (eu), the price is $6.25 per TiB.

### Capacity
BigQuery offers a capacity-based analysis pricing model for customers who prefer a predictable cost for queries rather than paying the on-demand price per TiB of data processed.

There are 3 editions *pricing as of October 2023:

#### Standard Edition
| Commitment model | Hourly cost | Details |
| - | - | - |
| Pay as you go / autoscale | \$0.066 / slot hour | Billed per second with a 1 minute minimum |

#### Enterprise Edition
Commitment model | Hourly cost | Details |
| - | - | - |
| Pay as you go / autoscale | \$0.066 / slot hour | Billed per second with a 1 minute minimum |
| 1 yr commit | \$0.0528 / slot hour | Billed for 1 year |
| 3 yr commit | \$0.0396 / slot hour | Billed for 3 years |

#### Enterprise Plus Edition
Commitment model | Hourly cost | Details |
| - | - | - |
| Pay as you go / autoscale | \$0.11 / slot hour | Billed per second with a 1 minute minimum |
| 1 yr commit | \$0.0.88 / slot hour | Billed for 1 year |
| 3 yr commit | \$0.0.66 / slot hour | Billed for 3 years |

When deciding between standard and enterprise, it’s important to note there are a few main differences. In the table below, the key advantages of Enterprise are listed. We would advise customers to only use Standard edition for basic ad hoc querying, trials and test projects.



Feature | Standard | Enterprise |
| - | - | - |
| Compute model | Autoscaling | Autoscaling + Baseline |
| Maximum reservation size | 1600 slots | None |
| VPC Service Controls | None | VPC Service Controls Support |
| Data governance	| None | Column-level access control, Row-level security, Dynamic data masking |
| Business Intelligence acceleration | None | BI Engine |
| Materialized views | Can query existing Materialized Views | Create materialized views, Automatic refresh of materialized views, Manual refresh of materialized views, Direct query of materialized views, Smart tuning |
| Integrated machine learning | None | BiqQuery ML |
| Workload management | Users cannot set the maximum concurrency target | Advanced workload management (idle capacity sharing, target concurrency) |

## This Notebook Setup
All results will be presented as graphs in this norkbook. It's necessary to first authenticate with Google and also clone the files necessary to output the results. 
New tables will be built from the INFOMATION_SCHEMA in a chosen GCP project. Note that there are 2 GCP projects identified in the notebook; they could be the same. However, when working with a 3rd party consultant from Google or otherwise, it is often the case that they will have a project set up to help with your cost analysis e.g. `consulant-project` that you give access to all your projects that you want to analyse. Then you can use the consulants project as the CONTROL_PROJECT_NAME. Usually when working on this internally, there is only one project for both CONTROL_PROJECT_NAME and INFOSCHEMA_PROJECT_NAME.

- The CONTROL_PROJECT_NAME is the project that will run the queries from this notebook and also the project where the tables we build will be stored under.
- The INFOSCHEMA_PROJECT_NAME is the project that we are using for the analysis. The INFOSCHEMA_PROJECT_NAME may have access to multiple projects' INFOMATION_SCHEMA. This can be expected and all accesible projects will appear in the analysis.

For example:
- There may be a control project called `consulant-project` which doesn't house any data that you use for analytics in your company, however it does have access to all the projects. This is the CONTROL_PROJECT_NAME which is essentially the billing project for the queries we will run below. This is also where the tables for analysis will be created.
- Then another project called `bigquery-cost-analysis` which also has access to all the other projects and is the INFOSCHEMA_PROJECT_NAME. This is what will be put in the FROM clause of the INFORMATION_SCEMA. ie `{INFOSCHEMA_PROJECT_NAME}`.`region-eu`.INFORMATION_SCHEMA.JOBS_BY_FOLDER.



Please run the next setup code blocks and authenticate as necessary.

In [ ]:
# @title Clone files from git
! git clone https://github.com/sam-pitcher/bq_cost_optimization_assessment.git

In [ ]:
# @title install dependancies from requirements.txt file
! pip install -r bq_cost_optimization_assessment/requirements.txt

In [ ]:
# Setup variables, please read carefully.

# We will build tables used for our analysis in BigQuery.
# These tables will need to be built in a project and a dataset.
# DESTINATION_PROJECT_NAME is the name of the project you want these tables to be built in.
DESTINATION_PROJECT_NAME = 'DESTINATION_PROJECT_NAME'
# DESTINATION_DATASET_NAME is the name of the dataset you want these tables to be built in.
# Ensure this dataset exists before running the code to build the tables, or you will receive an error.
DESTINATION_DATASET_NAME = 'slots_analysis'

# CONTROL_PROJECT_NAME is the project that will be querying the INFORMATION_SCHEMA.
# This will be used in the Authenication in the next step.
CONTROL_PROJECT_NAME = 'CONTROL_PROJECT_NAME'

# INFOSCHEMA_PROJECT_NAME is the project that will be in the from clause of the INFORMATION_SCHEMA.
# If you have set up a folder where all your projects you want to analyse reside, then set INFOSCHEMA_PROJECT_NAME as a project in that folder and the tables will be built for all projects in the same folder.
# You will need to ensure you have the bigquery.jobs.listAll permission at the folder level as either a custom role. Or use roles/bigquery.resourceViewer as this includes bigquery.jobs.listAll.
## INFOSCHEMA_PROJECT_NAME = "my_project"
INFOSCHEMA_PROJECT_NAME = "YOUR_PROJECT_NAME"
INFOSCHEMA_PROJECT_NAME_FOR_TABLE = INFOSCHEMA_PROJECT_NAME.replace("-", "_")
# Region for INFOSCHEMA_PROJECT_NAME
# Either "eu" or "us"
REGION = "eu"
# If folders are not set up, then the table will build for a single project. If this is the case, change PROJECT_OR_FOLDER to "project"
PROJECT_OR_FOLDER = "folder"

# SQL path for files from git.
# If running locally, the SQL_PATH is just "sql", otherwise use "bq_optimization_colab/sql".
# Comment out as appropriate.
SQL_PATH = "sql"
SQL_PATH = "bq_cost_optimization_assessment/sql"

START_DATE = "2024-01-01"
END_DATE = "2024-01-08"

In [ ]:
#@title Setup
from google.cloud import bigquery
import pandas
pandas.options.plotting.backend = "plotly"
import plotly.express as px
import numpy as np
try:
    from google.colab import auth
    from google.colab import data_table
    data_table.enable_dataframe_formatter()
    auth.authenticate_user()
except:
    pass

client = bigquery.Client(project=CONTROL_PROJECT_NAME, location=REGION)

def get_sql(sql_file_path, project_ids=[], time_range=""):
    INFO_TABLE_SUFFIX = ""
    if PROJECT_OR_FOLDER == "folder":
        INFO_TABLE_SUFFIX = "_BY_FOLDER"
    if len(project_ids) > 0:
        PROJECT_WHERE_CLAUSE = "AND project_id IN ('" + "' , '".join(project_ids) + "')"
    else:
        PROJECT_WHERE_CLAUSE = ""
    if len(time_range) > 0:
        PERIOD_START_WHERE_CLAUSE = f"AND TIMESTAMP_TRUNC(period_start, DAY) = TIMESTAMP('{time_range}')"
    else:
        PERIOD_START_WHERE_CLAUSE = ""
    sql = open(f"{SQL_PATH}/{sql_file_path}", "r").read()
    sql = sql.replace("{INFO_TABLE_SUFFIX}", INFO_TABLE_SUFFIX)
    sql = sql.replace("{INFOSCHEMA_PROJECT_NAME}", INFOSCHEMA_PROJECT_NAME)
    sql = sql.replace("{INFOSCHEMA_PROJECT_NAME_FOR_TABLE}", INFOSCHEMA_PROJECT_NAME_FOR_TABLE)
    sql = sql.replace("{CONTROL_PROJECT_NAME}", CONTROL_PROJECT_NAME)
    sql = sql.replace("{DESTINATION_PROJECT_NAME}", DESTINATION_PROJECT_NAME)
    sql = sql.replace("{DESTINATION_DATASET_NAME}", DESTINATION_DATASET_NAME)
    sql = sql.replace("{REGION}", REGION)
    # sql = sql.replace("{DAYS_AGO}", f"{DAYS_AGO}")
    sql = sql.replace("{START_DATE}", f"{START_DATE}")
    sql = sql.replace("{END_DATE}", f"{END_DATE}")
    # sql = sql.replace("{QUERIES_PER_DAY}", f"{QUERIES_PER_DAY}")
    sql = sql.replace("{PROJECT_WHERE_CLAUSE}", PROJECT_WHERE_CLAUSE)
    sql = sql.replace("{PERIOD_START_WHERE_CLAUSE}", PERIOD_START_WHERE_CLAUSE)
    try:
        sql = sql.replace("{slot_buckets_sql}", slot_buckets_case_when_sql)
    except:
        pass
    return sql

To remove the need of continually hitting the INFORMATION_SCHEMA, it's recommended to build some smaller tables with the necessary information from the INFORMATION_SCHEMA. These tables will then be used throughout the notebook.

In [ ]:
#@title Build Tables
# DAYS_AGO = 30
# QUERIES_PER_DAY = 200

client.query(get_sql("metric_table_builds/job_metrics.sql"))
client.query(get_sql("metric_table_builds/job_stages.sql"))
client.query(get_sql("metric_table_builds/timeline_metrics.sql"))
client.query(get_sql("metric_table_builds/timeline_metrics_by_job.sql"))

## BigQuery Pricing Plan Assistant
BigQuery information schema is holds the essential metadata for your BigQuery projects. It holds details about all your datasets, tables, views, jobs, and more, allowing you to explore and manage your data efficiently.

### Steps we need to address:
1. Cost effectiveness of measuring slots vs TBs processed
2. Compare Standard vs Enterprise
3. Compare Enterprise vs Enterprise Plus

<img src="bq_price_flowchart.png" width="50%">

It’s now important to include metrics about slot usage to understand if capacity pricing is cheaper.

In [ ]:
# @title What is the comparison in cost between on-demand and capacity?
# Either write a list of project ids as strings, or leave an empty list to include all projects
START_DATE = START_DATE #default value is "2024-01-01"
END_DATE = END_DATE #default value is "2024-01-08"

PROJECT_IDS = ["project_1", "project_2"]
PROJECT_IDS = []
capacity_cost_comparison_df = client.query(get_sql("capacity/cost_comparison.sql", PROJECT_IDS)).to_dataframe()
capacity_cost_comparison_df.head()

Splitting by project for projects that have >200 average queries per day. By default we have said that projects with under 200 queries per day are classed as ad-hoc, but this can change per customer. You can change the default below.

In [ ]:
# @title What is the comparison in cost between on-demand and capacity per project?
# Either write a list of project ids as strings, or leave an empty list to include all projects
PROJECT_IDS = ["project_1", "project_2"]
PROJECT_IDS = []
capacity_cost_comparison_per_project_df = client.query(get_sql("capacity/cost_comparison_per_project.sql", PROJECT_IDS)).to_dataframe()
capacity_cost_comparison_per_project_df.head()

Are projects using more than 1600 slots?

In [ ]:
# @title What is the consumption and slot usage across projects?
# Either write a list of project ids as strings, or leave an empty list to include all projects
PROJECT_IDS = ["project_1", "project_2"]
PROJECT_IDS = []
percentiles_df = client.query(get_sql("capacity/percentiles.sql", PROJECT_IDS)).to_dataframe()
percentiles_df.head()

In [ ]:
# @title What is the consumption and slot usage per project?
# Either write a list of project ids as strings, or leave an empty list to include all projects
PROJECT_IDS = ["project_1", "project_2"]
PROJECT_IDS = []
percentiles_per_project_df = client.query(get_sql("capacity/percentiles_per_project.sql", PROJECT_IDS)).to_dataframe()
percentiles_per_project_df.head()

As a heatmap it's easier to see

In [ ]:
percentiles_per_project_df.sort_values(by=['percentile95'], ascending=False).head(20).style.background_gradient(cmap='Blues')

It's valuable to see the slots used in buckets of time

In [ ]:
# @title What is the consumption and slot usage in buckets across projects?

# Either write a list of project ids as strings, or leave an empty list to include all projects
PROJECT_IDS = ["project_1", "project_2"]
PROJECT_IDS = []
slot_bucket_maximum = 10000
slot_buckets = np.linspace(0,slot_bucket_maximum,11)
slot_buckets = np.rint(slot_buckets).tolist()

slot_buckets_sort = []
slot_buckets_case_when_sql = """
case
"""
for i in range(11):
    try:
        slot_range = f'{i:02d}. {slot_buckets[i]} < {slot_buckets[i+1]}'
        slot_buckets_case_when_sql += f"""
        when total_slots between {slot_buckets[i]} and {slot_buckets[i+1]} then '{slot_range}'"""
    except:
        slot_buckets_case_when_sql += f"""
        when total_slots > {slot_buckets[i]} then '{i:02d}. > {slot_buckets[i]}'
        end as bucket
        """
    slot_buckets_sort.append(slot_range)

slot_buckets_df = client.query(get_sql("capacity/slot_buckets.sql", PROJECT_IDS)).to_dataframe()
slot_buckets_per_project_df = client.query(get_sql("capacity/slot_buckets_per_project.sql", PROJECT_IDS)).to_dataframe()

In [ ]:
slot_buckets_df.style.background_gradient(cmap='Blues', axis=0)

In [ ]:
slot_buckets_per_project_df.pivot(index='project_id', columns='bucket', values='pct').sort_values(by=slot_buckets_sort[::-1], ascending=False).head(20).style.background_gradient(cmap='Blues', axis=1)

In [ ]:
# Either write a list of project ids as strings, or leave an empty list to include all projects
PROJECT_IDS = ["project_1", "project_2"]
PROJECT_IDS = []
metrics_per_day_df = client.query(get_sql("analysis/metrics_per_day.sql", PROJECT_IDS)).to_dataframe()
# metrics_per_day_df.set_index('day', inplace=True)
metrics_per_day_df.head()

In [ ]:
# x_axis values: 'day', 'project_id'
x_axis = 'day'
# y_axis values: 'slot_ms', 'total_query_time', 'total_bytes_processed'
y_axis = 'slot_ms'
# kind values: 'area', 'bar', 'line'
kind = 'area'

plot_df = metrics_per_day_df[[x_axis,y_axis]].groupby(x_axis).sum().sort_values(by=[x_axis], ascending=[False])#.head(300)
plot_df.plot(kind=kind)

In [ ]:
# Either write a list of project ids as strings, or leave an empty list to include all projects
PROJECT_IDS = ["project_1", "project_2"]
PROJECT_IDS = []
TIME_RANGE = "2023-11-04"
metrics_per_second_df = client.query(get_sql("analysis/metrics_per_second.sql", PROJECT_IDS, TIME_RANGE)).to_dataframe()
# metrics_per_day_df.set_index('period_start', inplace=True)
metrics_per_second_df.head()

In [ ]:
# x_axis values: 'period_start', 'project_id', 'job_id'
x_axis = 'period_start'
# y_axis values: 'slots', 'total_bytes_processed'
y_axis = 'slots'
# kind values: 'area', 'bar', 'line'
kind = 'area'

plot_df = metrics_per_second_df[[x_axis,y_axis]].groupby(x_axis).sum().sort_values(by=[y_axis], ascending=[False])
# plot_df.head()
plot_df.plot(kind=kind)

## BigQuery Slow Query Optimiser
BigQuery information schema is holds the essential metadata for your BigQuery jobs. It holds details about all your datasets, tables, views, jobs, and more, allowing you to explore and manage your data efficiently.

### Steps we need to identify:
1. Slow running queries

In [ ]:
# Either write a list of project ids as strings, or leave an empty list to include all projects
PROJECT_IDS = ["project_1", "project_2"]
PROJECT_IDS = []
TIME_RANGE = "2023-11-04"
slow_queries_df = client.query(get_sql("queries/slow_queries.sql", PROJECT_IDS)).to_dataframe()
slow_queries_df_top_500 = slow_queries_df.head(500)
job_ids = "'"+"','".join(slow_queries_df_top_500['job_id'].head(50).to_list())+"'"

In [ ]:
slow_queries_df_top_500

In [ ]:
query = f"""
SELECT job_id,query FROM `dwh-ingestion-prod`.`region-eu`.INFORMATION_SCHEMA.JOBS
WHERE job_id IN
({job_ids})
"""
slow_queries_df = client.query(query).to_dataframe()
slow_queries_df

In [ ]:
# Plot slowest queries

# y_axis values: 'query_duration', 'duration', 'totalSlotUsage', 'peakSlotUsage', 'avgPeriodSlotUsage', 'totalProcessedTb'
y_axis = 'query_duration'
limit = 30

plot_df = slow_queries_df[["job_id",y_axis]].sort_values(by=[y_axis], ascending=[False]).head(limit)
fig = px.bar(plot_df, x="job_id", y=y_axis, color="job_id", title=f"Slowest {limit} queries").show()

In [ ]:
# Plot average value by dimension

# x_axis values: 'project_id', 'job_id', 'user_email'
x_axis = 'user_email'
# y_axis values: 'query_duration', 'duration', 'totalSlotUsage', 'peakSlotUsage', 'avgPeriodSlotUsage', 'totalProcessedTb'
y_axis = 'query_duration'
limit = 30

plot_df = slow_queries_df[[x_axis,y_axis]].groupby(x_axis, as_index=False).mean().sort_values(by=[y_axis], ascending=[False]).head(limit)
fig = px.bar(plot_df, x=x_axis, y=y_axis, color=x_axis, title=f"Average {y_axis} by {x_axis}").show()

In [ ]:
# Plot slowest 500 queries by project and job

fig = px.bar(slow_queries_df_top_500, x="project_id", y="query_duration", color="job_id", title="Slowest 500 queries by project and job").show()

## Reservations
First, it is important to see the slot usage for each reservation. We can use the pricing above to calculate what edition is best for the reservation below.

### Questions for Customer:
1. How did you initially distribute projects into reservations?
2. Are there specific ELT vs BI vs analytical projects?


In [ ]:
# @title Run query for TB Scanned
PROJECT_IDS = []
tb_scanned_df = client.query(get_sql("tb_scanned/tb_scanned.sql", PROJECT_IDS)).to_dataframe()
tb_scanned_df.head()

In [ ]:
# @title Run query for TB Scanned per Project
PROJECT_IDS = []
tb_scanned_per_project_df = client.query(get_sql("tb_scanned/tb_scanned_per_project.sql", PROJECT_IDS)).to_dataframe()
tb_scanned_per_project_df.head()

In [ ]:
# @title Run query for TB Scanned per Reservation
PROJECT_IDS = []
tb_scanned_per_reservation_df = client.query(get_sql("tb_scanned/tb_scanned_per_reservation.sql", PROJECT_IDS)).to_dataframe()
tb_scanned_per_reservation_df.head()